In [1]:
import pandas as pd
import numpy as np

%matplotlib inline 
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

#import sklearn
import sklearn.model_selection as ms
from sklearn.model_selection import train_test_split

# Featureset Scalers
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer


# Catboost model
import catboost as cb


# Grid Search and Cross Validation for Hyper-parameter tuning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#Model Eval Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error


# Examining feature importance
from sklearn.inspection import permutation_importance
import shap


# Style and plotting
 
# Set the style
plt.style.use('seaborn-v0_8-whitegrid')
# Display all columns
pd.set_option('display.max_columns', None)


/Users/eihwood/opt/miniforge3/envs/boost/lib/python3.11/site-packages/shap/utils/_clustering.py:34: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/Users/eihwood/opt/miniforge3/envs/boost/lib/python3.11/site-packages/shap/utils/_clustering.py:53: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/Users/eihwood/opt/miniforge3/envs/boost/lib/python3.11/site-p

# FUNCTION DEFINITION

In [80]:
# Define some function for evaluation and feature importance

def evaluate(model, X_train, X_test, y_train, y_test, logy=False):
    pred_dict = {
    'Train RMSE': [],
    'Test RMSE':[],
    'Train MAPE':[],
    'Test MAPE':[],
    'Train MAE':[],
    'Test MAE': [],
    'Train R2':[],
    'Test R2': []
    }
    

    
    r2_test = model.score(X_test, y_test)
    r2_train = model.score(X_train, y_train)
    pred_dict["Train R2"].append(r2_train)
    pred_dict["Test R2"].append(r2_test)

    
    # Train / Test Predictions for Base Model
    y_predict = model.predict(X_train)
    
    if logy:
        y_predict = np.exp(y_predict)
        y_test = np.exp(y_test)
        y_train = np.exp(y_train)
    
    #TRAIN
    #MSE = mean_squared_error(y_train, y_predict, squared=True)
    RMSE = mean_squared_error(y_train, y_predict, squared=False)
    MAE = mean_absolute_error(y_train, y_predict)
    MAPE = mean_absolute_percentage_error(y_train, y_predict)*100
    #print('MSE Train: ', MSE)
    print('RMSE Train: ', RMSE)
    print('MAE Train: ', MAE)
    print('MAPE Train: ', MAPE)
    
    pred_dict["Train RMSE"].append(RMSE)
    pred_dict["Train MAPE"].append(MAPE)
    pred_dict["Train MAE"].append(MAE)
    
    
    # TEST
    y_predict = model.predict(X_test)
    if logy:
        y_predict = np.exp(y_predict)
    #MSE = mean_squared_error(y_test, y_predict, squared=True)
    RMSE = mean_squared_error(y_test, y_predict, squared=False)
    MAE = mean_absolute_error(y_test, y_predict)
    MAPE = mean_absolute_percentage_error(y_test, y_predict)*100

    #print('MSE Val: ', MSE)
    print('RMSE Val: ', RMSE)
    print('MAE Val: ', MAE)
    print('MAPE Val: ', MAPE)
    pred_dict["Test MAE"].append(MAE)
    pred_dict["Test MAPE"].append(MAPE)
    pred_dict["Test RMSE"].append(RMSE)
    
    return pred_dict



## Data Loading and Feature, Target Partition

In [23]:
# Import Cleaned and Processed Data (no encoding or dummification)
data = pd.read_csv('../data/L1/ames_house_price_processed.csv', na_filter=False)

print(data.columns.to_series().groupby(data.dtypes).groups)

# Force categorical variables that are 'numeric' to be categorical
data["ms_sub_class"] = data['ms_sub_class'].astype('category')
data["overall_qual"] = data['overall_qual'].astype('category')
data["overall_cond"] = data['overall_cond'].astype('category')

# Define features and target
features = data.drop(['pid', 'sale_price', 'log_sale_price'], axis = 1)
target = data['log_sale_price']
logy = True # we are going with log(y) because our model sandbox demonstrated higher performance for logged depend.

from pandas.api.types import is_numeric_dtype  
def convert_cats(df):
        cats = []
        for col in df.columns:
            if is_numeric_dtype(df[col]):
                pass
            else:
                cats.append(col)
        for col in cats:
            df[col] = df[col].astype('category')

# Convert categorical variables in features
convert_cats(features)

# Get list of which columns are categorical
cols = features.columns
numeric_cols = features._get_numeric_data().columns

cat_cols1 = list(set(cols) - set(numeric_cols))
print(cat_cols1)

{int64: ['pid', 'gr_liv_area', 'sale_price', 'ms_sub_class', 'lot_area', 'overall_qual', 'overall_cond', 'central_air', '1st_flr_sf', '2nd_flr_sf', 'low_qual_fin_sf', 'full_bath', 'half_bath', 'bedroom_abv_gr', 'kitchen_abv_gr', 'tot_rms_abv_grd', 'fireplaces', 'wood_deck_sf', 'open_porch_sf', 'enclosed_porch', '3_ssn_porch', 'screen_porch', 'pool_area', 'misc_val', 'mo_sold', 'yr_sold', 'near_rr', 'near_main_rd', 'near_pos', 'house_age_at_sale', 'construction_age_at_sale'], float64: ['lot_frontage', 'mas_vnr_area', 'bsmt_fin_sf1', 'bsmt_fin_sf2', 'bsmt_unf_sf', 'total_bsmt_sf', 'bsmt_full_bath', 'bsmt_half_bath', 'garage_cars', 'garage_area', 'log_sale_price'], object: ['ms_zoning', 'alley', 'lot_shape', 'land_contour', 'lot_config', 'land_slope', 'neighborhood', 'bldg_type', 'house_style', 'roof_style', 'exterior1st', 'mas_vnr_type', 'exter_qual', 'exter_cond', 'foundation', 'bsmt_qual', 'bsmt_cond', 'bsmt_exposure', 'bsmt_fin_type1', 'bsmt_fin_type2', 'heating_qc', 'electrical', 'ki

## Train Test Split

In [4]:
# Train / test / split to isolate a test dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, \
                                                    test_size=0.10, shuffle = True, random_state = 8)



In [5]:

# Train / test split to isolate a validation dataset
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, \
                                                    test_size=0.10, random_state = 8)


print("X_train shape: {}".format(X_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("X_val shape: {}".format(X_val.shape))
print("y_train shape: {}".format(y_train.shape))
print("y_test shape: {}".format(y_test.shape))
print("y val shape: {}".format(y_val.shape))

X_train shape: (2088, 73)
X_test shape: (258, 73)
X_val shape: (232, 73)
y_train shape: (2088,)
y_test shape: (258,)
y val shape: (232,)


## CatBoost Recursive Feature Elimination

In [45]:
cbr = cb.CatBoostRegressor(eval_metric = 'RMSE',iterations = 100, \
                           cat_features=cat_cols1, random_state = 66, learning_rate = 0.1)

In [46]:
cbr.fit(X_train, y_train)

0:	learn: 0.3581779	total: 5.71ms	remaining: 566ms
1:	learn: 0.3338600	total: 11.3ms	remaining: 553ms
2:	learn: 0.3135490	total: 15.4ms	remaining: 499ms
3:	learn: 0.2945875	total: 18.9ms	remaining: 452ms
4:	learn: 0.2780207	total: 21.9ms	remaining: 416ms
5:	learn: 0.2633796	total: 24.7ms	remaining: 387ms
6:	learn: 0.2498689	total: 27.4ms	remaining: 364ms
7:	learn: 0.2373148	total: 29.9ms	remaining: 344ms
8:	learn: 0.2262258	total: 32.4ms	remaining: 328ms
9:	learn: 0.2162016	total: 37.1ms	remaining: 334ms
10:	learn: 0.2061914	total: 39.6ms	remaining: 321ms
11:	learn: 0.1976030	total: 42.3ms	remaining: 310ms
12:	learn: 0.1894464	total: 45ms	remaining: 301ms
13:	learn: 0.1822867	total: 47.6ms	remaining: 292ms
14:	learn: 0.1756761	total: 50.3ms	remaining: 285ms
15:	learn: 0.1696606	total: 54.7ms	remaining: 287ms
16:	learn: 0.1646409	total: 58ms	remaining: 283ms
17:	learn: 0.1602069	total: 60.6ms	remaining: 276ms
18:	learn: 0.1558794	total: 63.3ms	remaining: 270ms
19:	learn: 0.1518878	total

In [47]:
#EVALUATE FULL MODEL
print('Train R2:', cbr.score(X_train, y_train))
print('Validate R2:', cbr.score(X_val, y_val))
# EVAL
pred = evaluate(cbr, X_train, X_test, y_train, y_test, logy = True)
pred_df = pd.DataFrame(pred)
pred_df

Train R2: 0.9484030538284082
Validate R2: 0.928002884470312
MSE Train:  16777.67990669347
RMSE Train:  129.5286837217667
MAE Train:  11552.063415764742
MAPE Train:  6.557580026320946
MSE Val:  18689.268274194434
RMSE Val:  136.70869860471365
MAE Val:  13255.37781731752
MAPE Val:  9.296018334475695


,Train RMSE,Test RMSE,Train MAPE,Test MAPE,Train MAE,Test MAE,Train R2,Test R2
0,129.528684,136.708699,6.55758,9.296018,11552.063416,13255.377817,0.948403,0.870104


In [ ]:
cbr = cb.CatBoostRegressor(eval_metric = 'RMSE',iterations = 100, \
                           learning_rate = 0.1, cat_features=cat_cols, random_state = 123)

In [ ]:
# DEFINE RFE PROCESS
rfe_dict = cbr.select_features(X = X_train, 
                                     y = np.exp(y_train), 
                                     eval_set = (X_val,np.exp(y_val)), # Walkforward validation set we have created earlier
                                     features_for_select = '0-72', # Features that will be selected on the RFE
                                     num_features_to_select = 30, # Number of features to keep from the selected
                                     steps = 72, # Number of model iterations performed in the RFE
                                     verbose = 50, #
                                     train_final_model = False, # Train final model after RFE is finished
                                     plot = True # plot the ??? after the RFE is finished
                                     )

In [ ]:
# The above indicates a loss minimum of RMSE of around 18.5K at 28 removed features (73-28) selected features

In [ ]:
from catboost import Pool

train_pool = Pool(X_train, np.exp(y_train), cat_features=cat_cols)
val_pool = Pool(X_val, np.exp(y_val), cat_features=cat_cols)
test_pool = Pool(X_test, np.exp(y_test), cat_features=cat_cols)

from catboost import EShapCalcType, EFeaturesSelectionAlgorithm, CatBoostRegressor

def select_features_syntetic(algorithm: EFeaturesSelectionAlgorithm, steps: int = 73, select: int = 73-28):
    print('Algorithm:', algorithm)
    model = CatBoostRegressor(eval_metric = 'RMSE',iterations = 1000, \
                           learning_rate = 0.01, cat_features=cat_cols, random_state = 66)
    summary = model.select_features(
        train_pool,
        eval_set=val_pool,
        features_for_select='0-72',     # we will select from all features
        num_features_to_select=select,  # we want to select exactly important features
        steps=steps,                                     # more steps - more accurate selection
        algorithm=algorithm,
        shap_calc_type=EShapCalcType.Regular,            # can be Approximate, Regular and Exact
        train_final_model=True,                          # to train model with selected features
        logging_level='Silent',
        plot=True
    )
    print('Selected features indices:', summary['selected_features'])
    print('Selected features names:', summary['selected_features_names'])
    print('Eliminated features indices:', summary['eliminated_features'])
    print('Eliminated features names:', summary['eliminated_features_names'])
    
    return summary


In [ ]:
# run with 45 selected (28 removed)
synthetic_shap_summary = select_features_syntetic(algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues)


- results indicate that removing 18 (selecting 55) features results in the optimal minimization of the loss function. Thus, we will run this once more, selecting 55 features

In [ ]:
# The loss is really unstable after removing more than 20 features so we will conclude feature selection by choosing the best 20 to remove
synthetic_shap_summary = select_features_syntetic(algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues, \
                                                  steps=73-18, select = 73-18)


In [ ]:
synthetic_shap_summary.items()

In [6]:
#features_to_eliminate = synthetic_shap_summary['eliminated_features_names']

features_to_eliminate = ['fireplace_qu',
 'bsmt_qual',
 'lot_area',
 'sale_condition',
 'sale_type',
 'tot_rms_abv_grd',
 'full_bath',
 'misc_feature',
 'enclosed_porch',
 'garage_type',
 'bsmt_cond',
 'near_pos',
 'land_slope',
 'overall_cond',
 'exterior1st',
 'house_style',
 'mo_sold',
 'mas_vnr_type']


X_train_se = X_train.drop(features_to_eliminate, axis = 1)

X_val_se = X_val.drop(features_to_eliminate, axis = 1)
X_test_se = X_test.drop(features_to_eliminate, axis = 1)

# Get list of which columns are categorical
cols = X_train_se.columns
numeric_cols = X_train_se._get_numeric_data().columns

cat_cols = list(set(cols) - set(numeric_cols))
features_to_eliminate

['fireplace_qu',
 'bsmt_qual',
 'lot_area',
 'sale_condition',
 'sale_type',
 'tot_rms_abv_grd',
 'full_bath',
 'misc_feature',
 'enclosed_porch',
 'garage_type',
 'bsmt_cond',
 'near_pos',
 'land_slope',
 'overall_cond',
 'exterior1st',
 'house_style',
 'mo_sold',
 'mas_vnr_type']

## HYPER PARAMETER TUNING

Bayesian optimization

Random and grid search pay no attention to past results when searching the best hyperparametes. Bayesian optimization, in contrast to random or grid search, keeps track of past evaluation results which it uses to form a probabilistic model mapping hyperparameters to a probability of a score on the objective function. There're a number of libraries that can do it. 

From this tutorial: 
https://github.com/catboost/catboost/blob/93da9cea2e898560caa8a64ce4fe8d695752261e/catboost/tutorials/hyperparameters_tuning/hyperparameters_tuning_using_optuna_and_hyperopt.ipynb

In [8]:
import optuna

import optuna.distributions as od
from optuna.samplers import TPESampler
from catboost.utils import eval_metric
from sklearn.pipeline import Pipeline
#from sklearn.feature_selection import SelectKBest, chi2 # feature selection (18, 19, 20, 21) to remove so  




RANDOM_SEED = 79
EARLY_STOPPING_ROUND = 20 # Catboost will stop before reaching num_boost_round

In [63]:
from sklearn.model_selection import cross_val_score

#train_pool = Pool(data=X_train_se, label=y_train, cat_features=cat_cols)
#val_pool = Pool(data=X_val_se, label=y_val, cat_features=cat_cols)
def objective(trial):
    
    param = {
        "iterations": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100)
    }
    
    
#     param = {}
#     param['learning_rate'] = trial.suggest_discrete_uniform("learning_rate", 0.001, 0.02, 0.001)
#     param['depth'] = trial.suggest_int('depth', 9, 15)
#     param['l2_leaf_reg'] = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5)
#     param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
    
    
#     param['grow_policy'] = 'Depthwise'
#     param['iterations'] = 1000
#     param['use_best_model'] = True
#     param['eval_metric'] = 'MAE'
#     param['od_type'] = 'Iter'
#     param['od_wait'] = 20
#     param['random_state'] = RANDOM_SEED
#     param['logging_level'] = 'Silent'
    
    regressor = cb.CatBoostRegressor(**param, cat_features=cat_cols, silent=True)

    regressor.fit(X_train_se, y_train,
                  early_stopping_rounds=EARLY_STOPPING_ROUND)
    

    predictions = regressor.predict(X_val_se)
    rmse = mean_squared_error(np.exp(y_val), np.exp(predictions), squared=False)
    
    return rmse


In [55]:
# Create Study and optimize the optuna objective function
tune_se = optuna.create_study(study_name=f'catboost-seed--reduced18features', direction='minimize')
tune_se.optimize(objective, n_trials=200, n_jobs=-1, timeout=24000)


[I 2023-06-12 20:00:17,788] A new study created in memory with name: catboost-seed--reduced18features
[I 2023-06-12 20:00:18,216] Trial 5 finished with value: 0.20293458468451012 and parameters: {'learning_rate': 0.003169406932032436, 'depth': 1, 'subsample': 0.3110345126136329, 'colsample_bylevel': 0.1616092685606133, 'min_data_in_leaf': 74}. Best is trial 5 with value: 0.20293458468451012.
[I 2023-06-12 20:00:19,217] Trial 8 finished with value: 0.12765874120581966 and parameters: {'learning_rate': 0.004781954414371613, 'depth': 3, 'subsample': 0.736464099511869, 'colsample_bylevel': 0.06638354933101523, 'min_data_in_leaf': 6}. Best is trial 8 with value: 0.12765874120581966.
[I 2023-06-12 20:00:19,665] Trial 9 finished with value: 0.26719110476239893 and parameters: {'learning_rate': 0.0012086872240139723, 'depth': 2, 'subsample': 0.25457428486130856, 'colsample_bylevel': 0.05735828268717488, 'min_data_in_leaf': 93}. Best is trial 8 with value: 0.12765874120581966.
[I 2023-06-12 20:

[I 2023-06-12 20:01:51,651] Trial 23 finished with value: 0.10474507610217107 and parameters: {'learning_rate': 0.010209867545934823, 'depth': 7, 'subsample': 0.06292124395542126, 'colsample_bylevel': 0.9421779113589337, 'min_data_in_leaf': 47}. Best is trial 29 with value: 0.09978661302843006.
[I 2023-06-12 20:01:51,947] Trial 31 finished with value: 0.10179773076004589 and parameters: {'learning_rate': 0.04318302680456953, 'depth': 6, 'subsample': 0.16088733423938972, 'colsample_bylevel': 0.406400508856327, 'min_data_in_leaf': 83}. Best is trial 29 with value: 0.09978661302843006.
[I 2023-06-12 20:01:51,948] Trial 27 finished with value: 0.10218168229303487 and parameters: {'learning_rate': 0.012916655525188293, 'depth': 6, 'subsample': 0.3766589882942274, 'colsample_bylevel': 0.9967964801343234, 'min_data_in_leaf': 83}. Best is trial 29 with value: 0.09978661302843006.
[I 2023-06-12 20:01:52,528] Trial 32 finished with value: 0.1023828604575628 and parameters: {'learning_rate': 0.04

[I 2023-06-12 20:02:51,849] Trial 60 finished with value: 0.09942371264152522 and parameters: {'learning_rate': 0.05303619877042585, 'depth': 3, 'subsample': 0.37473061973466715, 'colsample_bylevel': 0.4656893478328775, 'min_data_in_leaf': 79}. Best is trial 33 with value: 0.09838801650665882.
[I 2023-06-12 20:02:52,387] Trial 58 finished with value: 0.10075026831542375 and parameters: {'learning_rate': 0.05281674327906526, 'depth': 3, 'subsample': 0.3662312246029409, 'colsample_bylevel': 0.6947254902635844, 'min_data_in_leaf': 86}. Best is trial 33 with value: 0.09838801650665882.
[I 2023-06-12 20:02:53,135] Trial 65 finished with value: 0.11069910057082337 and parameters: {'learning_rate': 0.039474194579934785, 'depth': 1, 'subsample': 0.4215254978980839, 'colsample_bylevel': 0.4649556150423355, 'min_data_in_leaf': 66}. Best is trial 33 with value: 0.09838801650665882.
[I 2023-06-12 20:02:54,545] Trial 64 finished with value: 0.10351665524154102 and parameters: {'learning_rate': 0.07

[I 2023-06-12 20:04:10,843] Trial 74 finished with value: 0.10954530288607542 and parameters: {'learning_rate': 0.04167214739204106, 'depth': 10, 'subsample': 0.294961594899741, 'colsample_bylevel': 0.7228845388624437, 'min_data_in_leaf': 82}. Best is trial 33 with value: 0.09838801650665882.
[I 2023-06-12 20:04:14,013] Trial 75 finished with value: 0.106343559943207 and parameters: {'learning_rate': 0.04180696231339241, 'depth': 10, 'subsample': 0.29171350281752867, 'colsample_bylevel': 0.6010711400398876, 'min_data_in_leaf': 82}. Best is trial 33 with value: 0.09838801650665882.
[I 2023-06-12 20:04:16,569] Trial 88 finished with value: 0.10024764711583402 and parameters: {'learning_rate': 0.03142113341259898, 'depth': 5, 'subsample': 0.2645964891758261, 'colsample_bylevel': 0.5332622713431966, 'min_data_in_leaf': 96}. Best is trial 33 with value: 0.09838801650665882.
[I 2023-06-12 20:04:16,953] Trial 89 finished with value: 0.10055049310514254 and parameters: {'learning_rate': 0.0317

[I 2023-06-12 20:05:52,747] Trial 117 finished with value: 0.10005540258566564 and parameters: {'learning_rate': 0.03481107741623295, 'depth': 5, 'subsample': 0.37992822009314403, 'colsample_bylevel': 0.7034268365389678, 'min_data_in_leaf': 79}. Best is trial 98 with value: 0.0975573561666987.
[I 2023-06-12 20:05:52,861] Trial 116 finished with value: 0.10015764295883998 and parameters: {'learning_rate': 0.027360601482473552, 'depth': 5, 'subsample': 0.3839497298313245, 'colsample_bylevel': 0.7016873568317585, 'min_data_in_leaf': 100}. Best is trial 98 with value: 0.0975573561666987.
[I 2023-06-12 20:05:56,895] Trial 118 finished with value: 0.10047262678322447 and parameters: {'learning_rate': 0.05679407797638799, 'depth': 5, 'subsample': 0.3620636284679829, 'colsample_bylevel': 0.7080073862232328, 'min_data_in_leaf': 84}. Best is trial 98 with value: 0.0975573561666987.
[I 2023-06-12 20:05:59,027] Trial 119 finished with value: 0.09950020377005735 and parameters: {'learning_rate': 0.

[I 2023-06-12 20:06:39,473] Trial 136 finished with value: 0.10326078760174727 and parameters: {'learning_rate': 0.03424938495586907, 'depth': 7, 'subsample': 0.29978583340935633, 'colsample_bylevel': 0.6368567403714193, 'min_data_in_leaf': 72}. Best is trial 98 with value: 0.0975573561666987.
[I 2023-06-12 20:06:39,702] Trial 143 finished with value: 0.10168299569374102 and parameters: {'learning_rate': 0.0220999021620279, 'depth': 6, 'subsample': 0.21789281389502752, 'colsample_bylevel': 0.5317708652556736, 'min_data_in_leaf': 72}. Best is trial 98 with value: 0.0975573561666987.
[I 2023-06-12 20:06:40,807] Trial 141 finished with value: 0.10203693608069601 and parameters: {'learning_rate': 0.03517959555646204, 'depth': 7, 'subsample': 0.3047884851005876, 'colsample_bylevel': 0.6533179280941009, 'min_data_in_leaf': 72}. Best is trial 98 with value: 0.0975573561666987.
[I 2023-06-12 20:06:42,465] Trial 145 finished with value: 0.1016648552406974 and parameters: {'learning_rate': 0.031

[I 2023-06-12 20:07:11,359] Trial 168 finished with value: 0.10165008572528109 and parameters: {'learning_rate': 0.029504946098694782, 'depth': 5, 'subsample': 0.2764004482451799, 'colsample_bylevel': 0.7302358686367344, 'min_data_in_leaf': 54}. Best is trial 157 with value: 0.09664845941727414.
[I 2023-06-12 20:07:11,678] Trial 170 finished with value: 0.09926360301133297 and parameters: {'learning_rate': 0.030043289060665942, 'depth': 5, 'subsample': 0.23797681798458908, 'colsample_bylevel': 0.7209717564553426, 'min_data_in_leaf': 49}. Best is trial 157 with value: 0.09664845941727414.
[I 2023-06-12 20:07:12,364] Trial 169 finished with value: 0.10041467009978897 and parameters: {'learning_rate': 0.030271187373076727, 'depth': 5, 'subsample': 0.1968138110855198, 'colsample_bylevel': 0.7288742282260917, 'min_data_in_leaf': 37}. Best is trial 157 with value: 0.09664845941727414.
[I 2023-06-12 20:07:15,900] Trial 173 finished with value: 0.10011122516901848 and parameters: {'learning_ra

[I 2023-06-12 20:07:47,560] Trial 197 finished with value: 0.1006199604473755 and parameters: {'learning_rate': 0.015915598110080444, 'depth': 6, 'subsample': 0.2584604402656491, 'colsample_bylevel': 0.6651990453596046, 'min_data_in_leaf': 46}. Best is trial 157 with value: 0.09664845941727414.
[I 2023-06-12 20:07:47,825] Trial 195 finished with value: 0.10160970578453643 and parameters: {'learning_rate': 0.02769266131757369, 'depth': 6, 'subsample': 0.25261097396218124, 'colsample_bylevel': 0.6675955865425185, 'min_data_in_leaf': 39}. Best is trial 157 with value: 0.09664845941727414.
[I 2023-06-12 20:07:47,931] Trial 196 finished with value: 0.10305982423065041 and parameters: {'learning_rate': 0.027589226314229512, 'depth': 6, 'subsample': 0.2538418622427782, 'colsample_bylevel': 0.6660403809263608, 'min_data_in_leaf': 47}. Best is trial 157 with value: 0.09664845941727414.
[I 2023-06-12 20:07:55,845] Trial 199 finished with value: 0.10412496248711091 and parameters: {'learning_rate

In [56]:
tune_se.best_params

{'learning_rate': 0.026028185473201213,
 'depth': 6,
 'subsample': 0.2198898716301622,
 'colsample_bylevel': 0.6659011781789096,
 'min_data_in_leaf': 40}

In [57]:
optuna.visualization.plot_optimization_history(tune_se).show(renderer="browser")

In [58]:
optuna.visualization.plot_param_importances(tune_se).show(renderer="browser")

In [59]:
# Train the final model using the best hyperparameters
final_model_se = cb.CatBoostRegressor(**tune_se.best_params, cat_features=cat_cols, eval_metric="MAE")
final_model_se.fit(X_train_se, y_train)

0:	learn: 0.2912892	total: 9.68ms	remaining: 9.67s
1:	learn: 0.2858019	total: 15.8ms	remaining: 7.87s
2:	learn: 0.2804179	total: 21.8ms	remaining: 7.25s
3:	learn: 0.2754398	total: 27.5ms	remaining: 6.84s
4:	learn: 0.2708135	total: 32.1ms	remaining: 6.4s
5:	learn: 0.2661043	total: 37.8ms	remaining: 6.27s
6:	learn: 0.2611257	total: 41.8ms	remaining: 5.93s
7:	learn: 0.2560228	total: 46.1ms	remaining: 5.72s
8:	learn: 0.2519805	total: 48.5ms	remaining: 5.34s
9:	learn: 0.2471202	total: 52.9ms	remaining: 5.24s
10:	learn: 0.2426998	total: 58.4ms	remaining: 5.25s
11:	learn: 0.2386066	total: 62.7ms	remaining: 5.17s
12:	learn: 0.2346812	total: 66.3ms	remaining: 5.03s
13:	learn: 0.2305195	total: 70.4ms	remaining: 4.96s
14:	learn: 0.2261419	total: 73.9ms	remaining: 4.85s
15:	learn: 0.2219208	total: 78.4ms	remaining: 4.82s
16:	learn: 0.2180078	total: 83.5ms	remaining: 4.83s
17:	learn: 0.2143075	total: 87ms	remaining: 4.75s
18:	learn: 0.2106797	total: 90.9ms	remaining: 4.69s
19:	learn: 0.2074799	tota

191:	learn: 0.0790042	total: 812ms	remaining: 3.42s
192:	learn: 0.0789172	total: 816ms	remaining: 3.41s
193:	learn: 0.0788716	total: 820ms	remaining: 3.4s
194:	learn: 0.0788342	total: 824ms	remaining: 3.4s
195:	learn: 0.0788105	total: 829ms	remaining: 3.4s
196:	learn: 0.0787115	total: 832ms	remaining: 3.39s
197:	learn: 0.0785937	total: 836ms	remaining: 3.39s
198:	learn: 0.0785021	total: 841ms	remaining: 3.38s
199:	learn: 0.0784468	total: 845ms	remaining: 3.38s
200:	learn: 0.0783867	total: 850ms	remaining: 3.38s
201:	learn: 0.0782739	total: 854ms	remaining: 3.37s
202:	learn: 0.0782054	total: 858ms	remaining: 3.37s
203:	learn: 0.0781107	total: 862ms	remaining: 3.36s
204:	learn: 0.0780110	total: 865ms	remaining: 3.35s
205:	learn: 0.0779613	total: 869ms	remaining: 3.35s
206:	learn: 0.0778605	total: 873ms	remaining: 3.34s
207:	learn: 0.0777711	total: 877ms	remaining: 3.34s
208:	learn: 0.0777293	total: 880ms	remaining: 3.33s
209:	learn: 0.0776771	total: 885ms	remaining: 3.33s
210:	learn: 0.0

376:	learn: 0.0698918	total: 1.62s	remaining: 2.68s
377:	learn: 0.0698641	total: 1.63s	remaining: 2.68s
378:	learn: 0.0698356	total: 1.63s	remaining: 2.67s
379:	learn: 0.0697963	total: 1.64s	remaining: 2.67s
380:	learn: 0.0697811	total: 1.64s	remaining: 2.66s
381:	learn: 0.0697636	total: 1.64s	remaining: 2.66s
382:	learn: 0.0697149	total: 1.65s	remaining: 2.65s
383:	learn: 0.0696639	total: 1.65s	remaining: 2.65s
384:	learn: 0.0696446	total: 1.65s	remaining: 2.64s
385:	learn: 0.0696177	total: 1.66s	remaining: 2.64s
386:	learn: 0.0696022	total: 1.66s	remaining: 2.63s
387:	learn: 0.0695228	total: 1.67s	remaining: 2.63s
388:	learn: 0.0694907	total: 1.67s	remaining: 2.63s
389:	learn: 0.0694355	total: 1.68s	remaining: 2.62s
390:	learn: 0.0694213	total: 1.68s	remaining: 2.62s
391:	learn: 0.0693986	total: 1.68s	remaining: 2.61s
392:	learn: 0.0693716	total: 1.69s	remaining: 2.61s
393:	learn: 0.0693491	total: 1.69s	remaining: 2.6s
394:	learn: 0.0693207	total: 1.7s	remaining: 2.6s
395:	learn: 0.0

583:	learn: 0.0632726	total: 2.42s	remaining: 1.73s
584:	learn: 0.0632494	total: 2.43s	remaining: 1.72s
585:	learn: 0.0632144	total: 2.43s	remaining: 1.72s
586:	learn: 0.0631987	total: 2.43s	remaining: 1.71s
587:	learn: 0.0631952	total: 2.44s	remaining: 1.71s
588:	learn: 0.0631768	total: 2.44s	remaining: 1.7s
589:	learn: 0.0631601	total: 2.45s	remaining: 1.7s
590:	learn: 0.0631406	total: 2.45s	remaining: 1.7s
591:	learn: 0.0631088	total: 2.45s	remaining: 1.69s
592:	learn: 0.0630846	total: 2.46s	remaining: 1.69s
593:	learn: 0.0630724	total: 2.46s	remaining: 1.68s
594:	learn: 0.0630377	total: 2.47s	remaining: 1.68s
595:	learn: 0.0630040	total: 2.47s	remaining: 1.68s
596:	learn: 0.0629657	total: 2.48s	remaining: 1.67s
597:	learn: 0.0629536	total: 2.48s	remaining: 1.67s
598:	learn: 0.0629376	total: 2.48s	remaining: 1.66s
599:	learn: 0.0629037	total: 2.49s	remaining: 1.66s
600:	learn: 0.0628832	total: 2.49s	remaining: 1.65s
601:	learn: 0.0628652	total: 2.5s	remaining: 1.65s
602:	learn: 0.06

787:	learn: 0.0587315	total: 3.22s	remaining: 866ms
788:	learn: 0.0587243	total: 3.22s	remaining: 862ms
789:	learn: 0.0587107	total: 3.23s	remaining: 858ms
790:	learn: 0.0586766	total: 3.23s	remaining: 854ms
791:	learn: 0.0586690	total: 3.23s	remaining: 849ms
792:	learn: 0.0586330	total: 3.24s	remaining: 845ms
793:	learn: 0.0586094	total: 3.24s	remaining: 841ms
794:	learn: 0.0585905	total: 3.25s	remaining: 837ms
795:	learn: 0.0585724	total: 3.25s	remaining: 833ms
796:	learn: 0.0585432	total: 3.25s	remaining: 829ms
797:	learn: 0.0585104	total: 3.26s	remaining: 825ms
798:	learn: 0.0585023	total: 3.26s	remaining: 821ms
799:	learn: 0.0584930	total: 3.27s	remaining: 817ms
800:	learn: 0.0584511	total: 3.27s	remaining: 813ms
801:	learn: 0.0584363	total: 3.27s	remaining: 808ms
802:	learn: 0.0584185	total: 3.28s	remaining: 804ms
803:	learn: 0.0583997	total: 3.28s	remaining: 800ms
804:	learn: 0.0583614	total: 3.28s	remaining: 796ms
805:	learn: 0.0583508	total: 3.29s	remaining: 792ms
806:	learn: 

991:	learn: 0.0544100	total: 4.01s	remaining: 32.4ms
992:	learn: 0.0543862	total: 4.02s	remaining: 28.3ms
993:	learn: 0.0543649	total: 4.02s	remaining: 24.3ms
994:	learn: 0.0543514	total: 4.03s	remaining: 20.2ms
995:	learn: 0.0543230	total: 4.03s	remaining: 16.2ms
996:	learn: 0.0543067	total: 4.03s	remaining: 12.1ms
997:	learn: 0.0542956	total: 4.04s	remaining: 8.09ms
998:	learn: 0.0542927	total: 4.04s	remaining: 4.05ms
999:	learn: 0.0542865	total: 4.05s	remaining: 0us


In [60]:
final_model_se.best_score_

{'learn': {'MAE': 0.054286490067159306, 'RMSE': 0.07246888638677378}}

In [62]:
evaluate(final_model_se, X_train_se, X_test_se, y_train, y_test, logy=True)

MSE Train:  14005.475724801558
RMSE Train:  118.34473256043785
MAE Train:  9880.08928631296
MAPE Train:  5.7455985777081295
MSE Val:  19140.49110669891
RMSE Val:  138.34916373689762
MAE Val:  13135.33584142903
MAPE Val:  9.448960471798914


{'Train RMSE': [118.34473256043785],
 'Test RMSE': [138.34916373689762],
 'Train MAPE': [5.7455985777081295],
 'Test MAPE': [9.448960471798914],
 'Train MAE': [9880.08928631296],
 'Test MAE': [13135.33584142903],
 'Train R2': [0.9601196891122048],
 'Test R2': [0.8599013216983318]}

# TUNE FULL MODEL WITH NO FEATURE REDUCTION AS COMPARISON

In [69]:
from sklearn.model_selection import cross_val_score

#train_pool = Pool(data=X_train_se, label=y_train, cat_features=cat_cols)
#val_pool = Pool(data=X_val_se, label=y_val, cat_features=cat_cols)
def objective(trial):
    
    param = {
        "iterations": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100)
    }
    
    
#     param = {}
#     param['learning_rate'] = trial.suggest_discrete_uniform("learning_rate", 0.001, 0.02, 0.001)
#     param['depth'] = trial.suggest_int('depth', 9, 15)
#     param['l2_leaf_reg'] = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5)
#     param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
    
    
#     param['grow_policy'] = 'Depthwise'
#     param['iterations'] = 1000
#     param['use_best_model'] = True
#     param['eval_metric'] = 'MAE'
#     param['od_type'] = 'Iter'
#     param['od_wait'] = 20
#     param['random_state'] = RANDOM_SEED
#     param['logging_level'] = 'Silent'
    
    regressor = cb.CatBoostRegressor(**param, cat_features=cat_cols1, silent=True)

    regressor.fit(X_train, y_train,
                  early_stopping_rounds=EARLY_STOPPING_ROUND)
    

    predictions = regressor.predict(X_val)
    rmse = mean_squared_error(np.exp(y_val), np.exp(predictions), squared=False)
    
    return rmse

# Create Study and optimize the optuna objective function
tune_full = optuna.create_study(study_name=f'catboost-seed--fullfeatures', direction='minimize')
tune_full.optimize(objective, n_trials=1000, n_jobs=-1, timeout=24000)

[I 2023-06-12 22:07:23,101] A new study created in memory with name: catboost-seed--fullfeatures
[I 2023-06-12 22:07:24,846] Trial 7 finished with value: 35208.01851301862 and parameters: {'learning_rate': 0.00226949320958069, 'depth': 4, 'subsample': 0.5286755725076551, 'colsample_bylevel': 0.0519674650804283, 'min_data_in_leaf': 90}. Best is trial 7 with value: 35208.01851301862.
[I 2023-06-12 22:07:25,444] Trial 2 finished with value: 19505.361929645383 and parameters: {'learning_rate': 0.06579348096308839, 'depth': 2, 'subsample': 0.33949099899030366, 'colsample_bylevel': 0.36974697406620344, 'min_data_in_leaf': 74}. Best is trial 2 with value: 19505.361929645383.
[I 2023-06-12 22:07:26,112] Trial 4 finished with value: 25229.726363337842 and parameters: {'learning_rate': 0.0035153926658792168, 'depth': 4, 'subsample': 0.44315767351113083, 'colsample_bylevel': 0.13563569260666614, 'min_data_in_leaf': 70}. Best is trial 2 with value: 19505.361929645383.
[I 2023-06-12 22:07:33,851] T

[I 2023-06-12 22:10:41,937] Trial 24 finished with value: 21377.994297174126 and parameters: {'learning_rate': 0.05612553728902747, 'depth': 10, 'subsample': 0.9760618079493346, 'colsample_bylevel': 0.9743077791759958, 'min_data_in_leaf': 61}. Best is trial 27 with value: 18858.009177641088.
[I 2023-06-12 22:10:51,432] Trial 22 finished with value: 20541.80940479958 and parameters: {'learning_rate': 0.07063987900795078, 'depth': 10, 'subsample': 0.9771297308700494, 'colsample_bylevel': 0.936490970247891, 'min_data_in_leaf': 55}. Best is trial 27 with value: 18858.009177641088.
[I 2023-06-12 22:10:55,212] Trial 30 finished with value: 18831.65756098153 and parameters: {'learning_rate': 0.050799919863109236, 'depth': 7, 'subsample': 0.6608727350262562, 'colsample_bylevel': 0.8583302783841547, 'min_data_in_leaf': 32}. Best is trial 30 with value: 18831.65756098153.
[I 2023-06-12 22:11:04,660] Trial 23 finished with value: 20047.32203059538 and parameters: {'learning_rate': 0.0663983239334

[I 2023-06-12 22:14:21,358] Trial 57 finished with value: 18990.82349712475 and parameters: {'learning_rate': 0.06720298146812383, 'depth': 6, 'subsample': 0.4917191136936668, 'colsample_bylevel': 0.9263763595923274, 'min_data_in_leaf': 36}. Best is trial 30 with value: 18831.65756098153.
[I 2023-06-12 22:14:22,916] Trial 58 finished with value: 19489.79084413324 and parameters: {'learning_rate': 0.03370307805196649, 'depth': 6, 'subsample': 0.5033478810069325, 'colsample_bylevel': 0.9188591547150371, 'min_data_in_leaf': 4}. Best is trial 30 with value: 18831.65756098153.
[I 2023-06-12 22:14:25,095] Trial 59 finished with value: 19462.906308058173 and parameters: {'learning_rate': 0.026316582613998312, 'depth': 6, 'subsample': 0.4728940917817821, 'colsample_bylevel': 0.7272812315754191, 'min_data_in_leaf': 4}. Best is trial 30 with value: 18831.65756098153.
[I 2023-06-12 22:14:30,329] Trial 66 finished with value: 20069.261034729196 and parameters: {'learning_rate': 0.02662008186400096

[I 2023-06-12 22:15:12,592] Trial 87 finished with value: 19130.191047460245 and parameters: {'learning_rate': 0.07797451065995052, 'depth': 4, 'subsample': 0.5974648872306318, 'colsample_bylevel': 0.6141157010065399, 'min_data_in_leaf': 9}. Best is trial 75 with value: 18146.509062254638.
[I 2023-06-12 22:15:12,993] Trial 92 finished with value: 18384.906106186016 and parameters: {'learning_rate': 0.08588560576990228, 'depth': 4, 'subsample': 0.32515847988008123, 'colsample_bylevel': 0.6030017143901076, 'min_data_in_leaf': 4}. Best is trial 75 with value: 18146.509062254638.
[I 2023-06-12 22:15:13,212] Trial 88 finished with value: 19020.471696321914 and parameters: {'learning_rate': 0.08162779117651324, 'depth': 4, 'subsample': 0.6713170851581461, 'colsample_bylevel': 0.6095980820542936, 'min_data_in_leaf': 9}. Best is trial 75 with value: 18146.509062254638.
[I 2023-06-12 22:15:13,264] Trial 74 finished with value: 19514.94749065021 and parameters: {'learning_rate': 0.07011997115700

[I 2023-06-12 22:15:42,386] Trial 111 finished with value: 18623.220813204487 and parameters: {'learning_rate': 0.06361367648446967, 'depth': 5, 'subsample': 0.6457531964332734, 'colsample_bylevel': 0.8960049281620722, 'min_data_in_leaf': 11}. Best is trial 95 with value: 18046.754058953127.
[I 2023-06-12 22:15:43,662] Trial 113 finished with value: 18971.716646060337 and parameters: {'learning_rate': 0.06491227921017675, 'depth': 5, 'subsample': 0.6575791690657394, 'colsample_bylevel': 0.5904481568794545, 'min_data_in_leaf': 7}. Best is trial 95 with value: 18046.754058953127.
[I 2023-06-12 22:15:44,969] Trial 118 finished with value: 18857.87700321056 and parameters: {'learning_rate': 0.06417158810220439, 'depth': 5, 'subsample': 0.3260025395765096, 'colsample_bylevel': 0.5859890855797076, 'min_data_in_leaf': 12}. Best is trial 95 with value: 18046.754058953127.
[I 2023-06-12 22:15:46,504] Trial 120 finished with value: 18749.527173760947 and parameters: {'learning_rate': 0.077875219

...

[I 2023-06-13 00:11:47,734] Trial 873 finished with value: 19824.92873580569 and parameters: {'learning_rate': 0.01662786617899339, 'depth': 4, 'subsample': 0.4119737595791985, 'colsample_bylevel': 0.710044138851026, 'min_data_in_leaf': 30}. Best is trial 283 with value: 17533.85983973332.
[I 2023-06-13 00:11:48,508] Trial 876 finished with value: 18348.362287796608 and parameters: {'learning_rate': 0.09186464868332571, 'depth': 4, 'subsample': 0.4103876445435561, 'colsample_bylevel': 0.7175105942604061, 'min_data_in_leaf': 33}. Best is trial 283 with value: 17533.85983973332.
[I 2023-06-13 00:11:49,182] Trial 877 finished with value: 19424.386161017137 and parameters: {'learning_rate': 0.0999239218589325, 'depth': 4, 'subsample': 0.38243492819653063, 'colsample_bylevel': 0.6424761102291646, 'min_data_in_leaf': 21}. Best is trial 283 with value: 17533.85983973332.
[I 2023-06-13 00:11:50,745] Trial 875 finished with value: 18299.435147975048 and parameters: {'learning_rate': 0.099814377

[I 2023-06-13 00:12:42,571] Trial 905 finished with value: 19114.68064091187 and parameters: {'learning_rate': 0.0916085444343175, 'depth': 4, 'subsample': 0.43936436489873565, 'colsample_bylevel': 0.9408448891791084, 'min_data_in_leaf': 31}. Best is trial 283 with value: 17533.85983973332.
[I 2023-06-13 00:12:48,762] Trial 906 finished with value: 19519.2208948567 and parameters: {'learning_rate': 0.03813110751480669, 'depth': 4, 'subsample': 0.4800928051978346, 'colsample_bylevel': 0.8889669095855066, 'min_data_in_leaf': 30}. Best is trial 283 with value: 17533.85983973332.
[I 2023-06-13 00:12:52,115] Trial 908 finished with value: 20435.37354364299 and parameters: {'learning_rate': 0.013241695304919308, 'depth': 5, 'subsample': 0.4878741395131726, 'colsample_bylevel': 0.500456317466003, 'min_data_in_leaf': 26}. Best is trial 283 with value: 17533.85983973332.
[I 2023-06-13 00:12:56,749] Trial 907 finished with value: 19247.173047315508 and parameters: {'learning_rate': 0.09033398625

[I 2023-06-13 00:14:16,547] Trial 928 finished with value: 18705.731609217186 and parameters: {'learning_rate': 0.05035286859641524, 'depth': 5, 'subsample': 0.6778992923414766, 'colsample_bylevel': 0.8845005708945813, 'min_data_in_leaf': 22}. Best is trial 283 with value: 17533.85983973332.
[I 2023-06-13 00:14:19,101] Trial 932 finished with value: 18744.939677177455 and parameters: {'learning_rate': 0.033610784974693754, 'depth': 5, 'subsample': 0.16009661526466396, 'colsample_bylevel': 0.6934371711403544, 'min_data_in_leaf': 31}. Best is trial 283 with value: 17533.85983973332.
[I 2023-06-13 00:14:19,472] Trial 933 finished with value: 22549.35983817845 and parameters: {'learning_rate': 0.0053309122465650454, 'depth': 4, 'subsample': 0.11670780196077243, 'colsample_bylevel': 0.999350297677848, 'min_data_in_leaf': 29}. Best is trial 283 with value: 17533.85983973332.
[I 2023-06-13 00:14:19,994] Trial 930 finished with value: 26859.50255095601 and parameters: {'learning_rate': 0.00264

[I 2023-06-13 00:15:08,442] Trial 955 finished with value: 18309.138087581476 and parameters: {'learning_rate': 0.08455544909117402, 'depth': 4, 'subsample': 0.38692148721292086, 'colsample_bylevel': 0.8594897362616507, 'min_data_in_leaf': 32}. Best is trial 283 with value: 17533.85983973332.
[I 2023-06-13 00:15:13,238] Trial 958 finished with value: 18790.64719286938 and parameters: {'learning_rate': 0.05530544460518239, 'depth': 4, 'subsample': 0.5827529347012664, 'colsample_bylevel': 0.862376956764079, 'min_data_in_leaf': 32}. Best is trial 283 with value: 17533.85983973332.
[I 2023-06-13 00:15:16,698] Trial 962 finished with value: 19067.650065696424 and parameters: {'learning_rate': 0.07298322947151392, 'depth': 3, 'subsample': 0.5666200826847079, 'colsample_bylevel': 0.8587940448317445, 'min_data_in_leaf': 24}. Best is trial 283 with value: 17533.85983973332.
[I 2023-06-13 00:15:18,569] Trial 964 finished with value: 18941.689802487446 and parameters: {'learning_rate': 0.07305501

[I 2023-06-13 00:16:09,685] Trial 983 finished with value: 19289.177800170593 and parameters: {'learning_rate': 0.07003138345080707, 'depth': 4, 'subsample': 0.635714215958765, 'colsample_bylevel': 0.9730707972398447, 'min_data_in_leaf': 22}. Best is trial 283 with value: 17533.85983973332.
[I 2023-06-13 00:16:10,453] Trial 986 finished with value: 18284.118837183618 and parameters: {'learning_rate': 0.0744330096852555, 'depth': 4, 'subsample': 0.5295805750747955, 'colsample_bylevel': 0.9410063618456009, 'min_data_in_leaf': 30}. Best is trial 283 with value: 17533.85983973332.
[I 2023-06-13 00:16:12,413] Trial 991 finished with value: 31396.85816483267 and parameters: {'learning_rate': 0.0021090131287671, 'depth': 5, 'subsample': 0.3187371327052099, 'colsample_bylevel': 0.35669472023404947, 'min_data_in_leaf': 25}. Best is trial 283 with value: 17533.85983973332.
[I 2023-06-13 00:16:14,909] Trial 987 finished with value: 22149.518288878076 and parameters: {'learning_rate': 0.0050919980

In [70]:
# EVALUATION
tune_full.best_params
optuna.visualization.plot_optimization_history(tune_full).show(renderer="browser")
optuna.visualization.plot_param_importances(tune_full).show(renderer="browser")

In [71]:
# Train the final full model using the best hyperparameters
final_model_full = cb.CatBoostRegressor(**tune_full.best_params, cat_features=cat_cols1, eval_metric="MAE")
final_model_full.fit(X_train, y_train)


0:	learn: 0.2829837	total: 6.79ms	remaining: 6.78s
1:	learn: 0.2673756	total: 13.6ms	remaining: 6.8s
2:	learn: 0.2535706	total: 17.9ms	remaining: 5.94s
3:	learn: 0.2409416	total: 22.6ms	remaining: 5.62s
4:	learn: 0.2289257	total: 25.8ms	remaining: 5.14s
5:	learn: 0.2190586	total: 29.5ms	remaining: 4.89s
6:	learn: 0.2091667	total: 33ms	remaining: 4.68s
7:	learn: 0.2003992	total: 36.5ms	remaining: 4.52s
8:	learn: 0.1917844	total: 41.1ms	remaining: 4.53s
9:	learn: 0.1847417	total: 45.3ms	remaining: 4.49s
10:	learn: 0.1770447	total: 48.6ms	remaining: 4.37s
11:	learn: 0.1703995	total: 52ms	remaining: 4.28s
12:	learn: 0.1643302	total: 56.1ms	remaining: 4.26s
13:	learn: 0.1581669	total: 59ms	remaining: 4.16s
14:	learn: 0.1523869	total: 62.4ms	remaining: 4.09s
15:	learn: 0.1479441	total: 66.1ms	remaining: 4.07s
16:	learn: 0.1434867	total: 69.4ms	remaining: 4.01s
17:	learn: 0.1398310	total: 73.6ms	remaining: 4.01s
18:	learn: 0.1365518	total: 77.1ms	remaining: 3.98s
19:	learn: 0.1327636	total: 8

166:	learn: 0.0700884	total: 607ms	remaining: 3.03s
167:	learn: 0.0699195	total: 610ms	remaining: 3.02s
168:	learn: 0.0698593	total: 614ms	remaining: 3.02s
169:	learn: 0.0698517	total: 617ms	remaining: 3.01s
170:	learn: 0.0697224	total: 620ms	remaining: 3.01s
171:	learn: 0.0697164	total: 624ms	remaining: 3s
172:	learn: 0.0696766	total: 627ms	remaining: 3s
173:	learn: 0.0696074	total: 631ms	remaining: 2.99s
174:	learn: 0.0695077	total: 634ms	remaining: 2.99s
175:	learn: 0.0694818	total: 639ms	remaining: 2.99s
176:	learn: 0.0693664	total: 642ms	remaining: 2.99s
177:	learn: 0.0693504	total: 646ms	remaining: 2.98s
178:	learn: 0.0692642	total: 649ms	remaining: 2.98s
179:	learn: 0.0692334	total: 653ms	remaining: 2.98s
180:	learn: 0.0691824	total: 657ms	remaining: 2.97s
181:	learn: 0.0691638	total: 660ms	remaining: 2.97s
182:	learn: 0.0691248	total: 663ms	remaining: 2.96s
183:	learn: 0.0691189	total: 667ms	remaining: 2.96s
184:	learn: 0.0689935	total: 671ms	remaining: 2.96s
185:	learn: 0.0689

329:	learn: 0.0618370	total: 1.2s	remaining: 2.44s
330:	learn: 0.0617671	total: 1.21s	remaining: 2.45s
331:	learn: 0.0617524	total: 1.21s	remaining: 2.44s
332:	learn: 0.0617284	total: 1.22s	remaining: 2.44s
333:	learn: 0.0617169	total: 1.22s	remaining: 2.44s
334:	learn: 0.0617028	total: 1.23s	remaining: 2.43s
335:	learn: 0.0615968	total: 1.23s	remaining: 2.43s
336:	learn: 0.0615639	total: 1.23s	remaining: 2.42s
337:	learn: 0.0614936	total: 1.24s	remaining: 2.42s
338:	learn: 0.0614909	total: 1.24s	remaining: 2.42s
339:	learn: 0.0614863	total: 1.24s	remaining: 2.41s
340:	learn: 0.0614657	total: 1.25s	remaining: 2.41s
341:	learn: 0.0614101	total: 1.25s	remaining: 2.41s
342:	learn: 0.0613603	total: 1.25s	remaining: 2.4s
343:	learn: 0.0613270	total: 1.26s	remaining: 2.4s
344:	learn: 0.0612596	total: 1.26s	remaining: 2.39s
345:	learn: 0.0612382	total: 1.26s	remaining: 2.39s
346:	learn: 0.0612303	total: 1.27s	remaining: 2.38s
347:	learn: 0.0611601	total: 1.27s	remaining: 2.38s
348:	learn: 0.0

495:	learn: 0.0560701	total: 1.8s	remaining: 1.83s
496:	learn: 0.0560139	total: 1.8s	remaining: 1.83s
497:	learn: 0.0559845	total: 1.81s	remaining: 1.82s
498:	learn: 0.0559620	total: 1.81s	remaining: 1.82s
499:	learn: 0.0558947	total: 1.82s	remaining: 1.82s
500:	learn: 0.0558887	total: 1.82s	remaining: 1.81s
501:	learn: 0.0558270	total: 1.82s	remaining: 1.81s
502:	learn: 0.0557940	total: 1.83s	remaining: 1.81s
503:	learn: 0.0557907	total: 1.83s	remaining: 1.8s
504:	learn: 0.0557818	total: 1.83s	remaining: 1.8s
505:	learn: 0.0557351	total: 1.84s	remaining: 1.79s
506:	learn: 0.0556988	total: 1.84s	remaining: 1.79s
507:	learn: 0.0556830	total: 1.84s	remaining: 1.79s
508:	learn: 0.0556753	total: 1.85s	remaining: 1.78s
509:	learn: 0.0556727	total: 1.85s	remaining: 1.78s
510:	learn: 0.0556706	total: 1.85s	remaining: 1.77s
511:	learn: 0.0556522	total: 1.86s	remaining: 1.77s
512:	learn: 0.0556459	total: 1.86s	remaining: 1.77s
513:	learn: 0.0556019	total: 1.87s	remaining: 1.76s
514:	learn: 0.05

661:	learn: 0.0522948	total: 2.4s	remaining: 1.23s
662:	learn: 0.0522396	total: 2.4s	remaining: 1.22s
663:	learn: 0.0522269	total: 2.41s	remaining: 1.22s
664:	learn: 0.0522147	total: 2.41s	remaining: 1.22s
665:	learn: 0.0521950	total: 2.42s	remaining: 1.21s
666:	learn: 0.0521513	total: 2.42s	remaining: 1.21s
667:	learn: 0.0521334	total: 2.42s	remaining: 1.2s
668:	learn: 0.0520926	total: 2.43s	remaining: 1.2s
669:	learn: 0.0520869	total: 2.43s	remaining: 1.2s
670:	learn: 0.0520673	total: 2.44s	remaining: 1.19s
671:	learn: 0.0520442	total: 2.44s	remaining: 1.19s
672:	learn: 0.0520338	total: 2.44s	remaining: 1.19s
673:	learn: 0.0520267	total: 2.45s	remaining: 1.18s
674:	learn: 0.0520176	total: 2.45s	remaining: 1.18s
675:	learn: 0.0519739	total: 2.45s	remaining: 1.18s
676:	learn: 0.0519565	total: 2.46s	remaining: 1.17s
677:	learn: 0.0519341	total: 2.46s	remaining: 1.17s
678:	learn: 0.0519140	total: 2.46s	remaining: 1.17s
679:	learn: 0.0518929	total: 2.47s	remaining: 1.16s
680:	learn: 0.051

823:	learn: 0.0489046	total: 3s	remaining: 641ms
824:	learn: 0.0488834	total: 3s	remaining: 637ms
825:	learn: 0.0488693	total: 3.01s	remaining: 634ms
826:	learn: 0.0488498	total: 3.01s	remaining: 630ms
827:	learn: 0.0488287	total: 3.02s	remaining: 626ms
828:	learn: 0.0487462	total: 3.02s	remaining: 623ms
829:	learn: 0.0487139	total: 3.02s	remaining: 619ms
830:	learn: 0.0486747	total: 3.03s	remaining: 616ms
831:	learn: 0.0486551	total: 3.03s	remaining: 612ms
832:	learn: 0.0486254	total: 3.03s	remaining: 608ms
833:	learn: 0.0486189	total: 3.04s	remaining: 604ms
834:	learn: 0.0486140	total: 3.04s	remaining: 601ms
835:	learn: 0.0485927	total: 3.04s	remaining: 597ms
836:	learn: 0.0485759	total: 3.05s	remaining: 594ms
837:	learn: 0.0485716	total: 3.05s	remaining: 590ms
838:	learn: 0.0485688	total: 3.05s	remaining: 586ms
839:	learn: 0.0485694	total: 3.06s	remaining: 583ms
840:	learn: 0.0485569	total: 3.07s	remaining: 580ms
841:	learn: 0.0485431	total: 3.07s	remaining: 576ms
842:	learn: 0.0485

In [81]:
print(final_model_full.best_score_)

evaluate(final_model_full, X_train, X_test, y_train, y_test, logy=True)

{'learn': {'MAE': 0.04576584628756357, 'RMSE': 0.060027348774606894}}
RMSE Train:  11366.626576481764
MAE Train:  8213.442964227143
MAPE Train:  4.776713459334203
RMSE Val:  17323.32413568199
MAE Val:  12168.876720968517
MAPE Val:  8.493950050035801


{'Train RMSE': [11366.626576481764],
 'Test RMSE': [17323.32413568199],
 'Train MAPE': [4.776713459334203],
 'Test MAPE': [8.493950050035801],
 'Train MAE': [8213.442964227143],
 'Test MAE': [12168.876720968517],
 'Train R2': [0.9734811594143148],
 'Test R2': [0.8821549101939576]}

In [ ]:
cross_val_score(estimator=final_model_full, )

In [83]:
preds = final_model_full.predict(features)

features['sales_price_predict'] = np.exp(preds)



features.to_csv('../data/L2/CatBoost_Sales_Price_Predictions_Full.csv', index=False)

In [84]:
features.head()

,gr_liv_area,ms_sub_class,ms_zoning,lot_frontage,lot_area,alley,lot_shape,land_contour,lot_config,land_slope,neighborhood,bldg_type,house_style,overall_qual,overall_cond,roof_style,exterior1st,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmt_fin_type1,bsmt_fin_sf1,bsmt_fin_type2,bsmt_fin_sf2,bsmt_unf_sf,total_bsmt_sf,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abv_gr,kitchen_abv_gr,kitchen_qual,tot_rms_abv_grd,functional,fireplaces,fireplace_qu,garage_type,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3_ssn_porch,screen_porch,pool_area,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,sale_condition,near_rr,near_main_rd,near_pos,house_age_at_sale,construction_age_at_sale,sales_price_predict
0,856,30,RL,80.0,7890,NA,Reg,Lvl,Corner,Gtl,SWISU,1Fam,1Story,6,6,Gable,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,TA,1,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,NA,NA,0,3,2010,WD,Normal,0,0,0,71,60,123474.615904
1,1049,120,RL,42.0,4235,NA,Reg,Lvl,Inside,Gtl,Edwards,TwnhsE,1Story,5,5,Gable,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,TA,1,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,NA,Attchd,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,NA,NA,0,2,2009,WD,Normal,0,0,0,25,25,133024.548370
2,1001,30,C (all),60.0,6060,NA,Reg,Lvl,Inside,Gtl,IDOTRR,1Fam,1Story,5,9,Hip,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,Ex,1,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,NA,Detchd,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,NA,NA,0,11,2007,WD,Normal,0,0,0,77,0,111847.007854
3,1039,70,RL,80.0,8146,NA,Reg,Lvl,Corner,Gtl,OldTown,1Fam,2Story,4,8,Gable,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,Gd,1,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,NA,Detchd,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,NA,NA,0,5,2009,WD,Normal,0,0,0,109,6,109593.125039
4,1665,60,RL,70.0,8400,NA,Reg,Lvl,Inside,Gtl,NWAmes,1Fam,2Story,8,6,Gable,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,643.0,Unf,0.0,167.0,810.0,Ex,1,SBrkr,810,855,0,1.0,0.0,2,1,3,1,Gd,6,Typ,0,NA,Attchd,Fin,2.0,528.0,TA,TA,Y,0,45,0,0,0,0,NA,NA,0,11,2009,WD,Normal,0,0,0,8,8,213097.937215


## Catboost Pipeline with Nested Cross Validation and Optuna Hyper-parameter tuning



In [ ]:
# COMPUTATIONALLY TOO EXPENSIVE

# import optuna
# from catboost import CatBoostRegressor
# from sklearn.model_selection import cross_val_score, KFold


# def objective_catboost(trial):
    
#     # Catboost parameters
#     param = {}       
#     param['learning_rate'] = trial.suggest_uniform("learning_rate", 0.001, 0.02, 0.1)
#     param['iterations'] = trial.suggest_int( "iterations", 100,1000,20)
#     param['depth'] = trial.suggest_int('depth', 9, 15)
#     param['l2_leaf_reg'] = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5)
#     param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
#     param['grow_policy'] = 'Depthwise'
#     param['use_best_model'] = True
#     param['eval_metric'] = 'MAE'
#     param['od_type'] = 'iter'
#     param['od_wait'] = 20
#     param['random_state'] = RANDOM_SEED
#     param['logging_level'] = 'Silent'
    
#     # Perform nested cross-validation
#     kf_outer = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
#     outer_scores = []
    
#     for train_index, val_index in kf_outer.split(X):
#         X_train, X_val = X[train_index], X[val_index]
#         y_train, y_val = y[train_index], y[val_index]
        
#          # Define the model with the current set of hyperparameters
#         model = CatBoostRegressor(**params)
        
#         # Perform hyperparameter tuning using inner cross-validation
#         kf_inner = KFold(n_splits=3, shuffle=True, random_state=42)
#         inner_scores = []
        
#         def inner_objective(trial_inner):
#             inner_params = {}
#             inner_params['learning_rate'] = trial.suggest_uniform("learning_rate", 0.001, 0.02, 0.1)
#             inner_params['iterations'] = trial.suggest_int( "iterations", 100,1000,20)
#             inner_params['depth'] = trial.suggest_int('depth', 9, 15)
#             inner_params['l2_leaf_reg'] = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5)
#             inner_params['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
#             inner_params['grow_policy'] = 'Depthwise'
#             inner_params['use_best_model'] = True
#             inner_params['eval_metric'] = 'MAE'
#             inner_params['od_type'] = 'iter'
#             inner_params['od_wait'] = 20
#             inner_params['random_state'] = RANDOM_SEED
#             inner_params['logging_level'] = 'Silent'
            
#             inner_model = CatBoostRegressor(**inner_params)
#             scores = cross_val_score(inner_model, X_train, y_train, cv=kf_inner)
#             return np.mean(scores)
        
#     # Optimize hyperparameters with Optuna
#     study_inner = optuna.create_study(direction='minimize')
#     study_inner.optimize(inner_objective, n_trials=100)
#     best_params = study_inner.best_params
        
#     # Train the model with the best hyperparameters
#     model = CatBoostRegressor(**best_params)
#     model.fit(X_train, y_train)
        
#     # Evaluate the model on the test set
#     score = model.score(X_val, y_val)
#     outer_scores.append(score)
    
#     # Calculate the mean score of outer cross-validation splits
#     mean_outer_score = np.mean(outer_scores)
#     return mean_outer_score

# Create the study object and optimize the objective function
#study = optuna.create_study(direction='minimize')
#study.optimize(objective, n_trials=500)
